# Barnard's Star

In [ ]:
import wobble
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
starid = 'barnards'
results = wobble.Results(filename='../results/results_'+starid+'_Kstar0_Kt0.hdf5')

In [ ]:
rvs = results.star_time_rvs + results.bervs

In [ ]:
sigs = np.ones_like(rvs) # HACK

In [ ]:
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
coord = SkyCoord(ra=269.4486, dec=4.7379807, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.01, u.degree)
height = u.Quantity(0.01, u.degree)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height);

In [ ]:
star = r[r['source_id'] == 4472832130942575872]

In [ ]:
star

In [ ]:
rv_gaia = -110.51 * u.km / u.s # from Gaia DR2 docs

In [ ]:
from astropy.time import Time
coord = SkyCoord(ra=star['ra'][0] * u.degree, 
                 dec=star['dec'][0] * u.degree, 
                 distance=1./star['parallax'][0] * u.kpc,
                 pm_ra_cosdec=star['pmra'][0] * u.mas / u.year,
                 pm_dec=star['pmdec'][0] * u.mas / u.year,
                 radial_velocity=rv,  
                 obstime=Time(star['ref_epoch'][0], format='decimalyear'),
                 frame='icrs')

In [ ]:
dates = Time(results.dates, format='jd')

In [ ]:
from tqdm import tqdm
rv_predicted = [coord.apply_space_motion(d).radial_velocity.value for d in tqdm(dates)]

In [ ]:
plt.plot(rv_predicted - np.median(rv_predicted))

In [ ]:
pm = np.sqrt(star['pmra'][0]**2 + star['pmdec'][0]**2)
pm = (pm * u.mas / u.yr).to(u.arcsec / u.yr)
pm

In [ ]:
dist = (1./star['parallax'][0] * u.kpc).to(u.pc)
dist

In [ ]:
vt = (4.74 * dist * pm).value * u.km / u.s # decompose doesn't work here

In [ ]:
min_dist = dist/np.sqrt(1. + rv_gaia**2/vt**2)

In [ ]:
min_dist

In [ ]:
v = np.sqrt(rv_gaia**2 + vt**2)
v

In [ ]:
obstime = Time(star['ref_epoch'][0], format='decimalyear')
min_dist_time = obstime + np.sqrt(dist**2 - min_dist**2)/v
min_dist_time

In [ ]:
def rv(date):
    t = (date - min_dist_time).jd * u.day # now relative to time of closest approach
    x = v**2 * t/np.sqrt(v**2 * t**2 + min_dist**2)
    return x.to(u.m / u.s)

def drv_dt(date):
    t = (date - min_dist_time).jd * u.day # now relative to time of closest approach
    x = v**2/np.sqrt(v**2 * t**2 + min_dist**2) - v**4 * t**2/(v**2 * t**2 + min_dist**2)**(3./2)
    return x.to(u.m / u.s / u.yr)

In [ ]:
date = dates[0]
t = (date - min_dist_time).jd * u.day
x = v**2 * t/np.sqrt(v**2 * t**2 + min_dist**2)
x.to(u.m / u.s)
rv(dates[20])

In [ ]:
predicted_rvs = np.array([rv(d).value for d in dates])

In [ ]:
pipeline_rvs = results.pipeline_rvs + results.bervs

In [ ]:
post_upgrade = results.dates >= 2457174.5 # June 2015
pre_upgrade = results.dates < 2457174.5

In [ ]:
plt.plot(results.dates[pre_upgrade], predicted_rvs[pre_upgrade] - np.mean(predicted_rvs[pre_upgrade]), c='b')
plt.plot(results.dates[pre_upgrade], rvs[pre_upgrade] - np.mean(rvs[pre_upgrade]), 'k.', alpha=0.5)
plt.xlabel('JD', fontsize=14)
plt.ylabel('RV (m/s)', fontsize=14);
#plt.plot(results.dates[post_upgrade], rvs[post_upgrade] - np.mean(rvs[post_upgrade]), 'k.', alpha=0.5);
#plt.plot(results.dates[pre_upgrade], pipeline_rvs[pre_upgrade] - np.mean(pipeline_rvs[pre_upgrade]), 'r.', alpha=0.5);

In [ ]:
wobble_resids = rvs[pre_upgrade] - predicted_rvs[pre_upgrade]
wobble_resids -= np.median(wobble_resids)
pipeline_resids = pipeline_rvs[pre_upgrade] - predicted_rvs[pre_upgrade]
pipeline_resids -= np.median(pipeline_resids)

wobble_resids_post = rvs[post_upgrade] - predicted_rvs[post_upgrade]
wobble_resids_post -= np.median(wobble_resids_post)
pipeline_resids_post = pipeline_rvs[post_upgrade] - predicted_rvs[post_upgrade]
pipeline_resids_post -= np.median(pipeline_resids_post)

In [ ]:
wobble_rms = np.sqrt(np.std(wobble_resids)**2 + np.std(wobble_resids_post)**2)
pipeline_rms = np.sqrt(np.std(pipeline_resids)**2 + np.std(pipeline_resids_post)**2)

In [ ]:
plt.plot_date(dates.plot_date[pre_upgrade], wobble_resids, 'k.')
plt.plot_date(dates.plot_date[post_upgrade], wobble_resids_post, 'k.');

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios':[4, 1]}, figsize=(8,6), sharex=True)
ax.plot_date(dates.plot_date[pre_upgrade], pipeline_resids, 'r.', alpha=0.6, 
             label='pipeline RV: RMS = {0:.2f} m/s'.format(pipeline_rms))
ax.plot_date(dates.plot_date[pre_upgrade], wobble_resids, 'k.', alpha=0.6, 
             label='wobble RV: RMS = {0:.2f} m/s'.format(wobble_rms))
ax.plot_date(dates.plot_date[post_upgrade], pipeline_resids_post, 'r.', alpha=0.6)
ax.plot_date(dates.plot_date[post_upgrade], wobble_resids_post, 'k.', alpha=0.6)
ax.set_title('Barnard\'s Star corrected for secular motion', fontsize=14)
ax.set_ylabel('RV (m/s)', fontsize=12)
ax2.set_xlabel('JD', fontsize=12)
ax2.plot_date(dates.plot_date[pre_upgrade], wobble_resids - pipeline_resids, 'k.')
ax2.plot_date(dates.plot_date[post_upgrade], wobble_resids_post - pipeline_resids_post, 'k.')
ax2.set_ylabel('Diff', fontsize=12)
ax2.set_ylim([-6,6])
ax.legend(fontsize=12)
fig.tight_layout();

In [ ]:
from astropy.stats import LombScargle
ls = LombScargle(results.dates[pre_upgrade], wobble_resids)
frequency, power = ls.autopower()

In [ ]:
probabilities = [0.05, 0.01, 0.001]
pow_5, pow_1, pow_01 = ls.false_alarm_level(probabilities) 

In [ ]:
plt.plot(1./frequency, power, c='k')
#plt.axhline(pow_5, c='r', ls='-.')
#plt.axhline(pow_1, c='r', ls=':')
#plt.axhline(pow_01, c='r', ls='--')
plt.xscale('log')
plt.xlabel('Period (d)', fontsize=14);